## Retrieval

First we have to load the corpus.

In [1]:
from datasets import load_dataset
meta_corpus = load_dataset(
    "json",
    data_files="data/corpus_chunks.jsonl",
    split="train"
).to_list()

In [2]:
import copy
import unicodedata as ud
import re
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from tqdm.notebook import tqdm
import string

def split_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = text.lower().split()
    words = [word for word in words if len(word.strip()) > 0]
    return words

## initiate BM25 retriever
tokenized_corpus = [split_text(doc["passage"]) for doc in tqdm(meta_corpus)]
bm25 = BM25Okapi(tokenized_corpus)

  0%|          | 0/48532 [00:00<?, ?it/s]

In [3]:
def retrieve(question, topk=50):
    """
    Get most relevant chunks to the question using simple BM25 score.
    """
    tokenized_query = split_text(question)
    bm25_scores = bm25.get_scores(tokenized_query)
    corpus_size = len(meta_corpus)
    for i in range(corpus_size):
        meta_corpus[i]["bm25_score"] = bm25_scores[i]
    bm25_passages = sorted(meta_corpus, key=lambda x: x["bm25_score"], reverse=True)
    return bm25_passages[:topk]

In [4]:
question = "Hồ Chí Minh là ai?"
top_passages = retrieve(question, topk=3)
top_passages

[{'title': 'Đội Thiếu niên Tiền phong Hồ Chí Minh',
  'passage': 'Title: Đội Thiếu niên Tiền phong Hồ Chí Minh\n\nThiếu niên Tiền phong Hồ Chí Minh. Tuyên ngôn hoạt động. Đội Thiếu niên Tiền phong Hồ Chí Minh là trường học giáo dục thiếu nhi Việt Nam trong và ngoài nhà trường, là đội dự bị của Đoàn Thanh niên Cộng sản Hồ Chí Minh; lực lượng nòng cốt trong các phong trào thiếu nhi. Đội viên. Đội viên Đội TNTP Hồ Chí Minh là thiếu niên Việt Nam từ 9 đến hết 15 tuổi thực hiện tốt 5 điều Bác Hồ dạy, phấn đấu học tập, rèn luyện và tu dưỡng để trở thành đoàn viên Đoàn TNCS Hồ Chí Minh. Lời hứa của đội viên. 1. Thực hiện tốt 5 điều Bác Hồ dạy thiếu niên, nhi đồng. 2. Tuân theo Điều lệ Đội Thiếu niên Tiền phong Hồ Chí Minh. 3. Giữ gìn danh dự Đội Thiếu niên Tiền phong Hồ Chí Minh. Quyền của đội',
  'id': 47894,
  'len': 159,
  'bm25_score': 19.956651855490467},
 {'title': 'Lực lượng Thanh niên xung phong Thành phố Hồ Chí Minh',
  'passage': 'Title: Lực lượng Thanh niên xung phong Thành phố Hồ 

The result is not accurate, showing the limit of BM25. Don't worry we will improve this in the `better_rag.ipynb`.

## Generation

In [5]:
prompt_template = (
    "### System:\n"
    "Below is an instruction that describes a task, paired with an input that provides further context. "
    "Write a response that appropriately completes the request.\n\n\n\n"
    "### Instruction:\n{instruction}\n\n"
    "### Input:\n{input}\n\n"
    "### Response:\n{output}"
)

def get_prompt(question, contexts):
    context = "\n\n".join([f"Context [{i+1}]: {x['passage']}" for i, x in enumerate(contexts)])
    instruction = 'You are an AI assistant. Provide a detailed answer so user don’t need to search outside to understand the answer.'
    input = f"Dựa vào một số ngữ cảnh được cho dưới đây, trả lời câu hỏi ở cuối.\n\n{context}\n\nQuestion: {question}\nHãy trả lời chi tiết và đầy đủ."
    prompt = prompt_template.format(
        instruction=instruction,
        input=input,
        output=''
    )
    return prompt

In [6]:
## Let's see how a prompt fed to the LLM looks like
contexts = retrieve(question, topk=3)
prompt = get_prompt(question, contexts)
print(prompt)

### System:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.



### Instruction:
You are an AI assistant. Provide a detailed answer so user don’t need to search outside to understand the answer.

### Input:
Dựa vào một số ngữ cảnh được cho dưới đây, trả lời câu hỏi ở cuối.

Context [1]: Title: Đội Thiếu niên Tiền phong Hồ Chí Minh

Thiếu niên Tiền phong Hồ Chí Minh. Tuyên ngôn hoạt động. Đội Thiếu niên Tiền phong Hồ Chí Minh là trường học giáo dục thiếu nhi Việt Nam trong và ngoài nhà trường, là đội dự bị của Đoàn Thanh niên Cộng sản Hồ Chí Minh; lực lượng nòng cốt trong các phong trào thiếu nhi. Đội viên. Đội viên Đội TNTP Hồ Chí Minh là thiếu niên Việt Nam từ 9 đến hết 15 tuổi thực hiện tốt 5 điều Bác Hồ dạy, phấn đấu học tập, rèn luyện và tu dưỡng để trở thành đoàn viên Đoàn TNCS Hồ Chí Minh. Lời hứa của đội viên. 1. Thực hiện tốt 5 điều Bác Hồ dạy thiếu niên, nhi đồng. 2. Tu

In [7]:
from transformers import GenerationConfig, TextStreamer
from transformers import LlamaForCausalLM, LlamaTokenizer, LlamaConfig
import torch

torch_dtype = torch.bfloat16
model_id = "llm4fun/vietrag-7b-v1.0"
device = "cuda"

tokenizer = LlamaTokenizer.from_pretrained(model_id)
model = LlamaForCausalLM.from_pretrained(
    model_id,
    config=LlamaConfig.from_pretrained(model_id),
    torch_dtype=torch_dtype
)

model = model.eval().to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
def generate(prompt, max_new_tokens=1024):
    """Text completion with a given prompt. In other words, give answer to your question.
    Args:
        prompt (str): Basically <instruction> + <question> + <retrieved_context>
        generation_config (not existed yet): For now, please manually tweak hyperparameters
        for generation in the `generation_config` below. Uncomment necessary arguments as you wish.
    Returns:
        list: an answer to the question within the prompt.
    """
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
    model.eval()
    with torch.no_grad():
        generation_config = GenerationConfig(
            repetition_penalty=1.13,
            max_new_tokens=max_new_tokens,
            # temperature=0.2,
            # top_p=0.95,
            # top_k=20,
            # bos_token_id=tokenizer.bos_token_id,
            # eos_token_id=tokenizer.eos_token_id,
            # eos_token_id=0, # for open-end generation.
            pad_token_id=tokenizer.pad_token_id,
            do_sample=False,
            use_cache=True,
            return_dict_in_generate=True,
            output_attentions=False,
            output_hidden_states=False,
            output_scores=False,
        )
        streamer = TextStreamer(tokenizer, skip_prompt=True)
        generated = model.generate(
            inputs=input_ids,
            generation_config=generation_config,
            streamer=streamer,
        )

    gen_tokens = generated["sequences"].cpu()[:, len(input_ids[0]):]
    output = tokenizer.batch_decode(gen_tokens)[0]
    output = output.split(tokenizer.eos_token)[0]
    return output.strip()

In [9]:
output = generate(prompt)

Ho Chi Minh là một nhà lãnh đạo cách mạng người Việt Nam, người đã dẫn dắt quốc gia này giành được độc lập từ Pháp vào năm 1945. Ông sinh ra với tên Nguyễn Ái Quốc và sau đó đổi thành Hồ Chí Minh. Ông là người sáng lập Đảng Cộng sản Việt Nam và là người đứng đầu nhà nước Việt Nam Dân chủ Cộng hòa từ năm 1945 đến năm 1969. Ông được coi là một trong những nhân vật quan trọng nhất trong lịch sử Việt Nam và được tôn vinh như một anh hùng dân tộc.</s>


## End-to-End RAG

It's almost done. Now let's try a simple RAG pipeline with our Wikipedia corpus.

In [10]:
def rag(question, topk=3):
    top_passages = retrieve(question, topk=topk)
    retrieved_context = "\n\n".join([f"Context [{i+1}]: {x['passage']}" 
                    for i, x in enumerate(top_passages)])
    prompt = get_prompt(question, contexts)
    output = generate(prompt)
    result = {
        "retrieved_context": retrieved_context,
        "generated_answer": output
    }
    return result

In [11]:
questions = [
    "Thành phố nào là thủ đô của Việt Nam?",
    "Samsung do ai sở hữu?",
    "Việt Nam có những ca sĩ KPOP nào?",
]
for question in questions:
    print(f"Câu hỏi: {question}")
    output = rag(question)
    print("---" * 30)

Câu hỏi: Thành phố nào là thủ đô của Việt Nam?
Thành phố Hồ Chí Minh là thủ đô của Việt Nam. Nó là một trong hai thành phố lớn nhất ở Việt Nam, cùng với Hà Nội. Thành phố có lịch sử lâu đời và là trung tâm văn hóa, kinh tế và chính trị của đất nước.</s>
------------------------------------------------------------------------------------------
Câu hỏi: Samsung do ai sở hữu?
Samsung Electronics Co., Ltd. là công ty con của Samsung Group, một tập đoàn quốc tế có trụ sở tại Hàn Quốc. Samsung Group được thành lập bởi Lee Byung-Chull vào năm 1938 và sau đó được chia thành hai công ty con riêng biệt: Samsung Electronics và Samsung Life Insurance. Samsung Electronics là công ty con lớn nhất của Samsung Group và là một trong những công ty lớn nhất thế giới, được điều hành bởi CEO Oh-Hyun Kwon.</s>
------------------------------------------------------------------------------------------
Câu hỏi: Việt Nam có những ca sĩ KPOP nào?
Việt Nam có một số ca sĩ K-pop, bao gồm Nguyễn Thành Phong, người 